# Introduction aux polynômes de chaos

TODO : obtenir la moyenne, la variance, les indices de Sobol' d'un chaos

TODO : estimation creuse par sélection de modèle

## Résumé

Dans cette page, nous présentons la décomposition en polynômes du chaos d'une fonction multivariée. Nous présentons les méthodes par intégration et par moindres carrés linéaires. Puis nous présentons le problème du sur-apprentissage ainsi que les méthodes de validation croisée. 

## Références

OpenTURNS :
* http://openturns.github.io/openturns/master/theory/meta_modeling/chaos_basis.html
* http://openturns.github.io/openturns/master/theory/meta_modeling/polynomial_least_squares.html
* http://openturns.github.io/openturns/master/theory/meta_modeling/cross_validation.html
* http://openturns.github.io/openturns/master/theory/meta_modeling/polynomial_sparse_least_squares.html

Ouvrages, articles, thèses :
* O.P Le Maître, O.M.Knio, Spectral methods for uncertainty quantification, Springer, 2010
* Wuan Luo. Wiener Chaos Expansion and Numerical Solutions of Stochastic Stochastic Partial Differential Equations. PhD thesis, California Institute of Technology, May 2006.
* R. H. Cameron and W. T. Martin. The orthogonal development of nonlinear functionals in series of Fourier-Hermite functionals. Annals of Mathematics, 48(2):385-392, April 1947.
* Adaptive sparse polynomial chaos expansions for uncertainty propagation and sensitivity analysis, Thèse, Géraud Blatman, 2009

Pour les plus aventuriers, vous pouvez prendre de risque de consulter mon propre texte qui contient la démonstration de plusieurs résultats présentés ici.
* Introduction to polynomials chaos with NISP. Michael Baudin. Version 0.5. March 2015

## Hypothèses

On considère une variable aléatoire standardisée $\xi\in\mathbb{R}^p$ où $n$ est la dimension de l'entrée. On considère la fonction $h$  :
$$
Y = g(\xi)
$$
où $Y\in\mathbb{R}$. 

*Note* : On considère ici le cas où la sortie $Y$ est un scalaire. Le cas plus général où la sortie $Y$ est un vecteur se traite de manière analogue, pour chaque composante du vecteur de sortie. Plus précisément, cela revient à créer plusieurs polynômes du chaos, chaque polynôme étant associé à une composante du vecteur de sortie. Cela ne nécessite pas nécessairement de nouveaux appels à la la fonction $g$, mais requiert des calculs supplémentaires. 

Supposons que :
* la variance de la sortie $Y$ est finie,
* les marginales de $\xi$ sont indépendantes. 

Sous ces hypothèses, notons $f_i$ la loi marginale de la variable aléatoire $\xi_i\in\mathbb{R}$, pour $i=1,...,n$. Puisque les variables sont indépentantes, la densité de probabilité du vecteur aléatoire $\xi$ est le produit des lois marginales, c'est à dire :
$$
f(\xi) = \prod_{i=1}^p f_i(\xi_i)
$$
pour tout $\xi\in\mathbb{R}^p$. 

## Décomposition en chaos polynomial

La décomposition précédente est une décomposition *spectrale*, analogue à la décomposition de Fourier. Lorsque la variable $X$ suit une loi gaussienne de marginales indépendantes, on utilise la base des polynômes d'Hermite multivariés (créés par tensorisation des polynômes d'Hermites univariés). Dans ce cas, cette décomposition s'appelle *décomposition en polynômes de chaos*. 

Dans le cas général où les marginales ne sont pas nécessairement gaussiennes, cette décomposition se nomme *décomposition en polynômes de chaos généralisés*. 

La preuve de la convergence des polynômes du cas (i.e. dans le cas particulier gaussien) est dûe, dans un cadre un peu plus général, à Cameron et Martin (voir (Le Maître, Knio, 2010) page 29). Le théorème de Cameron-Martin implique une variable aléatoire $\xi$ en dimension infinie, c'est à dire que $\xi=(\xi_1,\xi_2,...)$. 

Soit $\{\psi_j(\xi)\}_{j\geq 0}$ une famille de polynômes orthogonaux multivariés. On définit le produit scalaire entre deux polynômes : 
$$
\left( \psi_{j}(\xi),\psi_{k}(\xi) \right) = \mathbb{E} (\psi_{j}(\xi) \psi_{k}(\xi))
$$
pour $j,k\geq 1$. Par définition de l'espérance, on a :
$$
\left( \psi_{j}(\xi),\psi_{k}(\xi) \right) = \int_{\mathbb{R}^n} \psi_{j}(\xi) \psi_{k}(\xi) f(\xi) d\xi.
$$

On fait l'hypothèse que les polynômes multivariés $\psi_j$ sont orthogonaux, c'est à dire que le produit scalaire est égal à zéro pour deux polynômes différents :
$$
\left( \psi_j(\xi),\psi_k(\xi) \right) = 0,
$$
pour $j\neq k$.

Alors la sortie peut être décomposée sous la forme d'une combinaison linéaire des polynômes multivariés :
$$
Y = \sum_{j=0}^{\infty} a_j\psi_j(\xi),
$$
où $a_j\in\mathbb{R}$ sont les coefficients de la décomposition en polynômes du chaos et $\xi=(\xi_1,\xi_2,...)$ est une variable aléatoire en dimension infinie.

La première troncature consiste à restreindre le vecteur aléatoire $\xi=(\xi_1,...,\xi_p)$ à une dimension $p$ finie. 
La seconde troncature consiste à restreindre le nombre de termes dans la série, pour ne conserver que les $P$ premiers termes, où $P$ est un entier. 
Alors la sortie peut être décomposée sous la forme d'une combinaison linéaire des polynômes multivariés (voir (Le Maître, Knio, 2010) page 34) :
$$
Y = \sum_{j=0}^P a_j\psi_j(\xi) + \epsilon(p,P),
$$
où $\xi=(\xi_1,...,\xi_p)$, $\epsilon(p,P)$ est une variable aléatoire et 
$$
a_j=\frac{(g,\psi_j)}{\|\psi_j\|^2},
$$
pour $j\geq 0$.
De plus, la décomposition précédente converge dans le sens $L^2$ :
$$
\lim_{p,P\rightarrow \infty} \|\epsilon(p,P)\| = 0.
$$

**Proposition: Coefficients of the truncated decomposition**

Assume that the truncated decomposition is exact, i.e.
$$
g(\xi) = \sum_{j=0}^P a_j\psi_j(\xi)
$$
for any $\xi\in\mathbb{R}^p$.
Therefore, the truncated expansion is such that
$$
a_j=\frac{(g,\psi_j)}{\|\psi_j\|^2},
$$
for $k=0,...,P$.

**Preuve**
Indeed, for $k=0,...,P$, we have:

\begin{eqnarray*}
(g,\psi_k)
&=& \int_{\mathbb{R}^p} g(\xi) \psi_k(\xi) f(\xi)d\xi\\
&=& \int_{\mathbb{R}^p} \left(\sum_{j=0}^P a_j\psi_j(\xi)\right) \psi_k(\xi) f(\xi)d\xi\\
&=& \sum_{j=0}^{P} a_j \int_{\mathbb{R}^p} \psi_j(\xi) \psi_k(\xi) w(x)d\xi\\
&=& \sum_{j=0}^{P} a_j (\psi_j,\psi_k) \\
&=& a_k (\psi_k,\psi_k) \\
&=& a_k \|\psi_k\|^2,
\end{eqnarray*}

since $(\psi_j,\psi_k)=0$ if $j\neq k$.

Considérons maintenant le cas particulier où les polynômes multivariés $\psi_j$ sont orthonormaux. 
On note $\delta_{j,k}$ le symbole de Kronecker :
$$
\delta_{j,k} = 
\begin{cases} 
1 & \mbox{si } j=k,  \\ 
0 & \mbox{si } j \ne k
\end{cases}
$$
pour tout $j,k\geq 0$.
Par définition d'une famille de polynômes orthonormaux, on a 
$$
\left( \psi_j(\xi),\psi_k(\xi) \right) = \delta_{j,k},
$$
pour $j,k=0,...,P$.
Alors $\|\psi_j\|=1$ pour $j\geq 0$. 
Par conséquent, les coefficients de la décomposition en polynômes du chaos sont 
$$
a_j=(g,\psi_j),
$$
pour $j=0,...,P$.

## Moments et indices de sensibilité d'un chaos polynomial

Dans cette section, nous présentons comment calculer la moyenne, la variance et les indices de sensibilité du premier ordre et totaux d'un chaos polynomial.

Supposons que l'on a décomposé la fonction $g$ en chaos polynomial $\tilde{g}$. Si le chaos polynomial est une mauvaise approximation de la fonction, alors il ne peut être utilisé. C'est la raison pour laquelle le thème de la quantification de la qualité d'un métamodèle est un sujet important en pratique. Au contraire, si le métamodèle est une bonne approximation de la fonction $g$, alors on peut, en poussant notre avantage, substituer $\tilde{g}$ à la place de $g$. De cette manière, toutes les caractéristiques que nous souhaiterions connaître sur $g$, nous pouvons les tirer de $\tilde{g}$. 

La décomposition en chaos polynomial est la décomposition de Hoefding de la fonction. Ainsi, la plupart des propriétés de la variable aléatoire peuvent être directement déduites des coefficients $a\in\mathbb{R}^{P+1}$ où $P+1$ est le nombre de coefficients. En particulier, si la base polynomiale est orthonormée (et non pas seulement orthogonale), les caractéristiques sont particulièrement faciles à obtenir.

Supposons que la décomposition en chaos polynomiale est :
$$
\tilde{g}(\xi) = \sum_{j=0}^P a_j\psi_j(\xi)
$$
for any $\xi\in\mathbb{R}^p$.

La moyenne du chaos polynomial est 
$$
E\left[\tilde{g}(\xi)\right]= a_0,
$$
c'est à dire le premier coefficient de la décomposition.

De plus, 
$$
V\left[\tilde{g}(\xi)\right]=\sum_{j=1}^P a_j^2 \|\psi_j\|^2.
$$
Dans l'expression précédente, soulignons que la variance est une somme de carrés, à *l'exclusion* du terme $a_0$ qui n'apparaît pas. 
Si la base polynomiale est orthonormale, alors l'expression est particulièrement simple :
$$
V\left[\tilde{g}(\xi)\right]=\sum_{j=1}^P a_j^2.
$$

Pour calculer les indices de sensibilité, on observe que la variance $V\left[\tilde{g}(\xi)\right]$ est décomposée comme une somme de carrés par le chaos polynomial, chaque terme ne dépendant que de $\psi_j$. C'est pourquoi le calcul des indices de sensibilité est particulièrement simple, en ne considérant que les termes appropriés dans chaque cas. 

Pour exprimer les indices de sensibilité, il est plus facile de le faire en utilisant les multi-indices. 
Notons $\mathcal{J}$ l'ensemble des multi-indices de dimension $p$ :
$$
\mathcal{J}=\left\{\alpha=(\alpha_1,...,\alpha_p), \alpha_i\in\{0,1,2,...\}\right\}.
$$
Soit $\mathcal{J}_P$ l'ensemble des multi-indices associés à la troncature à l'ordre $P$ du chaos polynomial. 
Plus précisément, la taille de l'ensemble $\mathcal{J}_P$ est égale à $P+1$, c'est à dire que $card\left(\mathcal{J}_P\right)=P+1$. 
Supposons que la décomposition en chaos polynomiale est :
$$
\tilde{g}(\xi) = \sum_{\alpha\in\mathcal{J}_P} a_\alpha \psi_\alpha(\xi)
$$
pour tout $\xi\in\mathbb{R}^p$.

Pour $i=1,...,p$, on souhaite calculer l'indice de sensibilité du premier ordre $S_i$ et total $T_i$. 

Pour calculer $S_i$, il faut considérer l'ensemble des multi-indices $\alpha$ associés uniquement à la variable $\xi_i$. 
Ainsi, les polynômes considérés dépendent exclusivement de $\xi_i$ et pas des autres variables, ce qui permet de mesurer l'effet de la variable $\xi_i$ seule. 
Plus précisément, soit $\mathcal{J}_i^S$ l'ensemble des multi-indices tels que $\alpha_i>0$ et tel que les autres composantes du multi-indice sont nulles :
$$
\mathcal{J}_i^S=\left\{\alpha=(0,...0,\alpha_i,0,...,0)\in\mathcal{J}_P, \quad \alpha_i>0\right\}.
$$
Alors l'indice du premier ordre $S_i$ est la somme des carrés des coefficients associés à la i-ème variable :
$$
S_i = \frac{\sum_{\alpha\in\mathcal{J}_i^S} a_\alpha^2 \|\psi_\alpha\|^2}{V\left[\tilde{g}(\xi)\right]}.
$$

Pour calculer $T_i$, il faut considérer l'ensemble des multi-indices $\alpha$ associés à la variable $\xi_i$, y compris ses interactions avec les autres variables. 
Ainsi, les polynômes considérés dépendent tous de $\xi_i$ et, dans certains cas, également des autres variables, ce qui permet de mesurer l'effet de la variable $\xi_i$ y compris ses interactions. 
Plus précisément, soit $\mathcal{J}_i^T$ l'ensemble des multi-indices tels que $\alpha_i> 0$ :
$$
\mathcal{J}_i^T=\left\{\alpha=(\alpha_1,...,\alpha_i,...,\alpha_p)\in\mathcal{J}_P, \quad \alpha_i>0\right\}.
$$
Alors l'indice total $T_i$ est la somme des carrés des coefficients associés à la i-ème variable :
$$
T_i = \frac{\sum_{\alpha\in\mathcal{J}_i^T} a_\alpha^2 \|\psi_\alpha\|^2}{V\left[\tilde{g}(\xi)\right]}.
$$

On peut également obtenir des indices similaires, non plus seulement pour une variable donnée, mais pour un groupe de variables. Cette idée permet d'identifier, par exemple, quelles variables d'entrée peuvent être fixées

## Méthodes spectrales non intrusives

On appelle méthodes spectrales non intrusives (NISP) les méthodes qui considèrent la fonction comme une boîte noire. L'objectif de ces méthodes est de calculer le vecteur des coeffients $a\in\mathbb{R}^{P+1}$ de la décomposition en chaos polynomial. 

Il y a deux grandes familles de méthodes pour calculer les coefficients de la décomposition en polynômes de chaos.
* Les méthodes par intégration consistent à évaluer l'intégrale associée au produit scalaire définissant le coefficient $a_j$. 
* Les méthodes par moindres carrés linéaires consistent à calculer le vecteur des coefficients en résolvant un problème de moindres carrés linéaires dans lequel la base de fonctions est la base polynomiale multivariées.

## Méthodes par intégration

Dans les méthodes par intégration, on évalue l'intégrale :
$$
a_j=(g,\psi_j) = \int_{\mathbb{R}^p} g(\xi)\psi_j(\xi)f(\xi)d\xi,
$$
pour $j=0,1,...,P$.

Plusieurs méthodes sont envisageables pour estimer la valeur de l'intégrale.
* La méthode la plus simple repose sur l'intégration par la méthode de Monte-Carlo simple, mais toute méthode d'échantillonage pour estimer une intégrale est élligible, comme l'utilisation d'un plan LHS ou une séquence à faible discrépance par exemple. 
* On peut également utiliser les méthodes de quadrature de Gauss multidimensionnelles par tensorisation. Toutefois, les méthodes de tensorisation de quadrature de Gauss sont associés à un grand nombre de noeuds, ce qui mène à un grand nombre d'évaluations de la fonction $g$. 
* Pour limiter la difficulté, les méthodes de quadratures imbriquées et de grilles creuses (*sparse grids* en anglais) permettent de réduire le nombre de noeuds d'intégration (voir (Le Maître, Knio, 2010) pages 51-63). 

L'inconvénient des méthodes déterministes (comme la quadrature de Gauss par exemple) pour l'estimation des intégrales requises est que le nombre d'évaluations de la fonction $g$ est fixé à l'avance par l'algorithme. Ce n'est pas très pratique, surtout dans les situations où on souhaite ajouter des points dans le plan d'expériences pour améliorer la qualité du métamodèle. Dans un tel contexte, il peut être intéressant d'envisager les méthodes d'estimations par moindres carrés, présentées dans la prochaine section.

## Méthodes par moindres carrés

Dans les méthodes par moindres carrés linéaires les plus simples, on considère un plan d'expériences de type Monte-Carlo simple. Soit $n\in\mathbb{N}$ est un entier représentant la taille de l'échantillon. Soit $\left\{x^{(j)}\in\mathbb{R}^p\right\}_{j=1,...,n}$ un échantillon i.i.d. du vecteur aléatoire $X$. 
Par conséquent, la réalisation $x_i^{(j)}$ est la i-ème composante de la j-ème réalisation, pour $i=1,...,p$ et $j=1,...,n$. 

Soit $y\in\mathbb{R}^n$ le vecteur des observations de la sortie :
$$
y^{(j)} = g\left(x^{(j)}\right),
$$
pour $j=1,...,n$.

Pour calculer les variables standardisées $\left\{\xi^{(j)}\right\}_{j=1,...,n}$, on considère l'équation 
$$
\xi_i^{(j)} = T_i\left(X_i^{(j)}\right),
$$
pour $i=1,\dots,p$ et $j=1,...,n$.

La matrice de conception $Z\in\mathbb{R}^{n\times (P+1)}$ définie par :
$$
z_{ji} = \psi_i\left(\xi^{(j)}\right)
$$
pour $i=0,\dots,P+1$ et $j=1,...,n$. 
Alors le vecteur des prédictions du chaos polynomial est $Za$ où $a\in\mathbb{R}^{P+1}$ est 
le vecteur des coeffients  de la décomposition en chaos polynomial. 
La méthode des moindres carrés linéaires consiste à minimiser la norme euclidienne des écarts entre les observations du modèle et les prédictions du chaos polynomial:
$$
\hat{a} = \textrm{argmin}_{a\in\mathbb{R}^{P+1}} \|y - Za\|_2,
$$
où $\|\cdot\|_2$ est la norme euclidienne en dimension $n$. 
La solution est donnée par les équations normales 
$$
\hat{a} = \left(Z^T Z\right)^{-1} Z^T y.
$$

## Surapprentissage et chaos polynomial

Sur la base de la fonction $g$, l'étape d'apprentissage a produit un métamodèle $\tilde{g}$. 
Si nous connaissions la fonction $g$, nous pourrions calculer l'erreur quadratique moyenne $Err$ définie par :
$$
Err = E\left[\left(g(X)-\tilde{g}(X)\right)^2\right].
$$

Malheureusement, estimer $Err$ peut nécessiter un grand nombre d'évaluations de $g$, peut être plus que pour estimer les coefficients du métamodèle $\tilde{g}$.
Pour contourner ce problème, considèrons le coefficient $R^2$ pour estimer la qualité du métamodèle sur la base d'apprentissage (d'où l'indice "a"). 
Supposons que le plan d'expériences $\left\{x^{(j)}_a\right\}_{j=1,...,n}$ est utilisé pour estimer les coefficients d'un modèle de regression linéaire tel qu'un métamodèle de chaos polynomial par exemple. 
$$
R^2(g(x_a),\tilde{g}(x_a)) = 1 - \frac{ \sum_{j=1}^N \left( y^{(j)}_a - \tilde{y}^{(j)}_a \right)^2  }{ \sum_{j=1}^N \left( y^{(j)}_a - \bar{y}_a \right)^2 }
$$
où $\bar{y}_a = \frac{1} {N} \sum_{i=1}^N y^{(j)}_a$.

La difficulté est la suivante: quand le nombre de coefficients dans la décomposition augmente, le $R^2$ diminue. La difficulté essentielle est que, même si le $R^2$ est petit, le métamodèle peut ne pas être de qualité suffisante (voir (Blatman, 2009) page 83). Le problème de la généralisation du métamodèle consiste à obtenir un métamodèle de qualité.  

Supposons qu'un second plan d'expériences de validation (d'où l'indice "v") du métamodèle est généré : $\left\{x^{(j)}_v\right\}_{j=1,...,n}$. L'objectif de ce second plan d'expériences est de tester le métamodèle sur des points d'entrées que le métamodèle n'a pas considéré pour estimer ses coefficients.

Soient $g(x_v)$ et $\tilde{g}(x_v)$ les sorties du modèle et du métamodèle sur le plan d'expériences de validation. 
Dans ce contexte, le coefficient $Q^2$ est égal au coefficient $R^2$ appliqué au plan d'expériences de validation :
$$
Q^2 = R^2(g(x_v),\tilde{g}(x_v)).
$$

Il s'avère que, en général, lorsque le nombre de coefficients dans le métamodèles augmente, le $R^2$ diminue tout le temps, tandis que le $Q^2$ diminue d'abord, puis augmente. 

D'un côté, le coefficient $R^2$ ne permet pas de mesurer efficacement la prédictivité du métamodèle. D'un autre côté, le coefficient $Q^2$ permet de bien quantifier la prédictivité, mais nécessite des nouvelles évaluations de la fonction $g$ sur un plan d'expériences de validation. Une méthode plus robuste d'estimation de l'erreur de validation que le $R^2$ et moins coûteuse que le $Q^2$ est la méthode *leave-one-out*, présentée dans la suite.

Le problème du surapprentissage est lié au nombre d'observations et au nombre de coefficients : pour obtenir un métamodèle de qualité, il est préférable d'avoir un nombre relativement faible de coefficients par rapport au nombre d'observations. La difficulté est que, pour obtenir un chaos polynomial dont la prédictivité est élevée, il est parfois nécessaire d'augmenter le degré du polynôme. En conséquence, le nombre de coefficients augmente, ce qui provoque un surapprentissage. Un premier moyen pour limiter ce problème consiste à utiliser une règle d'énumération creuse qui favorise des termes élevés sans trop augmenter le nombre de coefficients liés aux interactions entre les variables. Un second moyen pour limiter le problème consiste à utiliser une méthode de sélection de modèle, ce qui est présenté dans la section suivante.

## K-fold et leave-one-out

La méthode de validation croisée consiste à découper l'échantillon en deux, le premier étant utilisé pour l'estimation des coefficients et le second pour la validation. Cette méthode a l'inconvénient de laisser de côté une grande partie de l'échantillon, qui n'est pas utilisée pour l'apprentissage. Pour limiter le problème, un entier $K$ étant donné, la méthode du K-Fold est un raffinement qui consiste à découper l'échantillon en K sous-échantillons de tailles presques égales. L'échantillon d'apprentissage est alors constitué de tous les sous-échantillons sauf un, qui est utilisé pour la validation. La méthode leave-one-out (LOO) peut alors être vue comme un cas particulier de K-Fold avec K=1. 

Dans la méthode classique, on considère un plan d'expériences 
$$
\mathcal{X} = \left\{x^{(j)}\in\mathbb{R}^p\right\}_{j=1,...,n}
$$
de taille $n$ et on estime les coefficients du métamodèle $\tilde{g}$. Dans la méthode leave-one-out, pour $j=1,...,n$, on considère le plan d'expériences $\mathcal{X}^{(-j)}$ dans lequel la j-ème expérience a été retirée :
$$
\mathcal{X}^{(-j)} = \left\{x^{(1)},...,x^{(j-1)},x^{(j+1)},...,,x^{(n)}\in\mathbb{R}^p\right\}_{j=1,...,n}.
$$
Notons $\tilde{g}^{(-j)}$ le métamodèle dont les coefficients ont étés appris sur le plan d'expériences $\mathcal{X}^{(-j)}$. 
L'erreur de prédiction au point $x^{(j)}$ est alors la différence entre la prédiction du modèle $g$ et la prédiction du métamodèle LOO $\tilde{g}^{(-j)}$ :
$$
\Delta^{(j)} = g\left(x^{(j)}\right) - \tilde{g}^{(-j)}\left(x^{(j)}\right)
$$
pour $j=1,...,n$. En d'autres termes, on évalue le résidu pour un point $x^{(j)}$ qui n'a pas été observé par le métamodèle LOO $\tilde{g}^{(-j)}$. 

L'erreur LOO est alors la moyenne empirique des erreurs :
$$
E_{LOO} = \frac{1}{n} \sum_{j=1}^n \left(\Delta^{(j)}\right)^2.
$$

Le coefficient de prédictivité $Q^2$ est alors calculé par :
$$
Q^2 = 1 - \frac{E_{LOO}}{\hat{V}(Y)}
$$
où $\hat{V}(Y)$ est la variance du vecteur des sorties. 
En pratique, cette variance est estimée sur la base de l'échantillon. 
Remarquons que, lorsque l'erreur LOO $E_{LOO}$ augmente, alors le $Q^2$ diminue.

La difficulté de l'expression de l'erreur de prédiction LOO est qu'elle nécessite de calculer les coefficients du métamodèle leave-one-out $\tilde{g}^{(-j)}$, ce qui peut être coûteux. 
Soit $H$ la matrice chapeau des moindres carrés linéaires :
$$
H = Z (Z^T Z) Z^T.
$$
Pour un modèle de regression linéaire comme le chaos polynomial par moindres carrés linéaires, la matrice de projection $H$ est la matrice qui projette les observations $y$ en les prédictions $\hat{y} = H y$. On peut démontrer que l'erreur LOO est égale à :
$$
\Delta^{(j)} = \frac{g\left(x^{(j)}\right) - \tilde{g}\left(x^{(j)}\right)}{1-h_{jj}}
$$
où $h_{jj}$ est le j-ème terme diagonal de la matrice chapeau $H$. 

En pratique, l'expression précédente est estimée en substituant le métamodèle $\tilde{g}$ à la place du modèle $g$. 

Le coefficient $Q^2$ précédent peut être corrigé pour tenir compte du nombre de paramètres estimés dans le modèle. L'objectif de cette erreur LOO corrigée est d'augmenter l'erreur lorsque le nombre de paramètres augmente. On considère alors l'erreur *ajustée* définie par :
$$
E_{LOO}^\star = E_{LOO} \frac{n-1}{n-P-1}.
$$
L'erreur LOO *corrigée* est une variante permettant de prendre en compte les situations où la taille de l'échantillon est petite. 

Plus de détails sur ce thème sont donnés dans (Blatman, 2009) pages 85 et 86.

## Méthodes de sélection de modèle

TODO